In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections

### Instacart dataset description
`orders` (3.4m rows, 206k users):
* `order_id`: order identifier
* `user_id`: customer identifier
* `eval_set`: which evaluation set this order belongs in (see `SET` described below)
* `order_number`: the order sequence number for this user (1 = first, n = nth)
* `order_dow`: the day of the week the order was placed on
* `order_hour_of_day`: the hour of the day the order was placed on
* `days_since_prior`: days since the last order, capped at 30 (with NAs for `order_number` = 1)

`products` (50k rows):
* `product_id`: product identifier
* `product_name`: name of the product
* `aisle_id`: foreign key
* `department_id`: foreign key

`aisles` (134 rows):
* `aisle_id`: aisle identifier
* `aisle`: the name of the aisle

`deptartments` (21 rows):
* `department_id`: department identifier
* `department`: the name of the department

`order_products__SET` (30m+ rows):
* `order_id`: foreign key
* `product_id`: foreign key
* `add_to_cart_order`: order in which each product was added to cart
* `reordered`: 1 if this product has been ordered by this user in the past, 0 otherwise

where `SET` is one of the four following evaluation sets (`eval_set` in `orders`):
* `"prior"`: orders prior to that users most recent order (~3.2m orders)
* `"train"`: training data supplied to participants (~131k orders)
* `"test"`: test data reserved for machine learning competitions (~75k orders)

In [13]:
aisles_path='data/instacart_2017_05_01/aisles.csv'
dep_path='data/instacart_2017_05_01/departments.csv'
orders_path='data/instacart_2017_05_01/orders.csv'
products_path='data/instacart_2017_05_01/products.csv'
o_ppath='data/instacart_2017_05_01/order_products__prior.csv'

In [14]:
aisles=pd.read_csv(aisles_path)
dep=pd.read_csv(dep_path)
orders=pd.read_csv(orders_path)
products=pd.read_csv(products_path)
oprior=pd.read_csv(o_ppath)

In [15]:
products.product_id.is_unique

True

In [16]:
orders.order_id.is_unique

True

In [17]:
products=products.set_index('product_id')

In [18]:
print(oprior.shape)
oprior.head()

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


**Aggregate and save prior orders**

Better to load the data the aggregation takes time

In [ ]:
oprior_agg=oprior.groupby('order_id').agg(list)

In [ ]:
oprior_agg.head()

In [ ]:
oprior_agg.to_pickle('Data/instacart_2017_05_01/oprior_agg.pkl')

**Load the order agregated data**

In [ ]:
oprior_agg=pd.read_pickle('Data/instacart_2017_05_01/oprior_agg.pkl')

In [ ]:
# query all the products associated with the order_id=6
products.loc[oprior_agg.product_id[6]]

In [ ]:
oprior.head()

In [ ]:
tmp=oprior.groupby('product_id').count()[['order_id']]
tmp.columns=['n_orders']
products=products.merge(tmp,left_index=True, right_index=True)

In [ ]:
products=products.sort_values('n_orders',ascending=False)

In [ ]:
products.head(n=2500)

In [ ]:
products.quantile(0.95) #only 5% of the products have more than 2286 orders we can focus on those products

In [ ]:
prod_95=products[products['n_orders']>=products.quantile(0.95)['n_orders']].copy()

In [ ]:
sns.distplot(np.log(products['n_orders'].values))
plt.xlabel('Log of the number or orders')
plt.show()

In [ ]:
prod_95.head()